## Importing Data

In [1]:
### Import data
import h5py
import numpy as np

# Specify file path
file_path = r"C:\Users\nnaga\Downloads\bird_spectrograms.hdf5"

# Open the file
f = h5py.File(file_path, 'r')

# Print keys (12 species)
print(list(f.keys()))  

['amecro', 'amerob', 'bewwre', 'bkcchi', 'daejun', 'houfin', 'houspa', 'norfli', 'rewbla', 'sonspa', 'spotow', 'whcspa']


### Binary Classification: Test Models

In [2]:
### Saving smaller sample for testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np

# Define file and species
species1 = 'amecro'
species2 = 'whcspa'
samples_per_class = 30  # Smaller sample for testing

# Extract and transpose a limited number of samples
X1 = np.transpose(f[species1][:, :, :samples_per_class], (2, 0, 1))  # (30, 128, 517)
X2 = np.transpose(f[species2][:, :, :samples_per_class], (2, 0, 1))  # (30, 128, 517)

# Create feature and label arrays
X = np.concatenate((X1, X2), axis=0)  # Shape: (60, 128, 517)
y = np.concatenate((np.ones(X1.shape[0]), np.zeros(X2.shape[0])))  # Shape: (60,)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=13)

# Convert labels to binary class format
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

# Check final shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(40, 128, 517) (40, 1)
(20, 128, 517) (20, 1)


In [3]:
### Optimal Model -------------------------------------------------------------------------
### Define Nerual Network Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    # detects patterns in images
        # 32 filters, 3x3 kernel, ReLU activation, input shape (height, width, channel)
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    # reduce dimensionality
    MaxPooling2D(pool_size=(2, 2)),

    # more complex features (increase to 64 filters)
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # converts multi dimensional space to 1D vector
    Flatten(),
    # hidden layer
    Dense(64, activation='relu'),
    # dropout later 50% nodes randomly dropped
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output
])

### Compile Neural Network Model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train Neural Network Model
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50


C:\Users\nnaga\miniconda3\envs\myenvname\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 379ms/step - accuracy: 0.4167 - loss: 335.8305 - val_accuracy: 0.5000 - val_loss: 313.2868
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.5625 - loss: 183.6291 - val_accuracy: 0.5000 - val_loss: 25.5856
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - accuracy: 0.4375 - loss: 73.0363 - val_accuracy: 0.5000 - val_loss: 74.2484
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - accuracy: 0.5000 - loss: 68.9358 - val_accuracy: 0.5000 - val_loss: 4.0409
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.5625 - loss: 9.2363 - val_accuracy: 0.5000 - val_loss: 21.6716
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - accuracy: 0.5208 - loss: 10.4941 - val_accuracy: 0.7500 - val_loss: 0.5568
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.8125 - loss: 1.0439 - val_accuracy: 0.6250 - val_loss: 0.4777
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.8958 - loss: 0.2758 - val_accuracy: 0.5000 - val_loss

In [4]:
### Higher Convolutional Layers -------------------------------------------------------------------------
### Define Nerual Network Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    # detects patterns in images
        # 32 filters, 3x3 kernel, ReLU activation, input shape (height, width, channel)
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    # reduce dimensionality
    MaxPooling2D(pool_size=(2, 2)),

    # more complex features (increase to 64 filters)
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # more complex features (increase to 128 filters)
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),


    # converts multi dimensional space to 1D vector
    Flatten(),
    # hidden layer
    Dense(64, activation='relu'),
    # dropout later 50% nodes randomly dropped
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output
])

### Compile Neural Network Model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train Neural Network Model
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 464ms/step - accuracy: 0.5833 - loss: 160.3083 - val_accuracy: 0.5000 - val_loss: 131.2066
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.5000 - loss: 87.1581 - val_accuracy: 0.5000 - val_loss: 0.6816
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step - accuracy: 0.4375 - loss: 0.9514 - val_accuracy: 0.5000 - val_loss: 0.6855
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.4792 - loss: 1.8176 - val_accuracy: 0.5000 - val_loss: 0.6758
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 357ms/step - accuracy: 0.5000 - loss: 0.8350 - val_accuracy: 0.3750 - val_loss: 0.6934
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 323ms/step - accuracy: 0.5000 - loss: 0.6933 - val_accuracy: 0.3750 - val_loss: 0.6934
Test Accuracy: 45.00%


In [5]:
### Smaller Kernel -------------------------------------------------------------------------
### Define Nerual Network Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    # detects patterns in images
        # 32 filters, 2x2 kernel, ReLU activation, input shape (height, width, channel)
    Conv2D(32, (2, 2), activation='relu', input_shape=(128, 517, 1)),
    # reduce dimensionality
    MaxPooling2D(pool_size=(2, 2)),

    # more complex features (increase to 64 filters)
    Conv2D(64, (2, 2), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # converts multi dimensional space to 1D vector
    Flatten(),
    # hidden layer
    Dense(64, activation='relu'),
    # dropout later 50% nodes randomly dropped
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output
])

### Compile Neural Network Model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train Neural Network Model
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 369ms/step - accuracy: 0.3958 - loss: 598.5447 - val_accuracy: 0.5000 - val_loss: 51.2918
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.5625 - loss: 69.2561 - val_accuracy: 0.5000 - val_loss: 195.4090
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.4167 - loss: 143.7131 - val_accuracy: 0.5000 - val_loss: 51.2604
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step - accuracy: 0.5625 - loss: 69.2797 - val_accuracy: 0.5000 - val_loss: 16.9202
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - accuracy: 0.5000 - loss: 16.6135 - val_accuracy: 0.6250 - val_loss: 0.6054
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.3958 - loss: 69.5148 - val_accuracy: 0.5000 - val_loss: 1.6665
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.4792 - loss: 1.7231 - val_accuracy: 0.5000 - val_loss: 0.6922
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.5417 - loss: 19.3491 - val_accuracy: 0.50

In [6]:
### Reduce Dropout -------------------------------------------------------------------------
### Define Nerual Network Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    # detects patterns in images
        # 32 filters, 3x3 kernel, ReLU activation, input shape (height, width, channel)
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    # reduce dimensionality
    MaxPooling2D(pool_size=(2, 2)),

    # more complex features (increase to 64 filters)
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # more complex features (increase to 128 filters)
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),


    # converts multi dimensional space to 1D vector
    Flatten(),
    # hidden layer
    Dense(128, activation='relu'),
    # dropout later 50% nodes randomly dropped
    Dropout(0.4),
    Dense(1, activation='sigmoid')  # Binary output
])

### Compile Neural Network Model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train Neural Network Model
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step - accuracy: 0.4792 - loss: 330.6016 - val_accuracy: 0.5000 - val_loss: 595.0089
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 318ms/step - accuracy: 0.5208 - loss: 292.6227 - val_accuracy: 0.5000 - val_loss: 66.2463
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 312ms/step - accuracy: 0.5625 - loss: 29.6536 - val_accuracy: 0.5000 - val_loss: 19.1794
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 323ms/step - accuracy: 0.4375 - loss: 13.4317 - val_accuracy: 0.5000 - val_loss: 25.8764
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 332ms/step - accuracy: 0.5208 - loss: 14.4486 - val_accuracy: 0.5000 - val_loss: 7.1211
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 343ms/step - accuracy: 0.5000 - loss: 5.8783 - val_accuracy: 0.5000 - val_loss: 6.2183
Test Accuracy: 50.00%


### Multi- Class Classification

In [7]:
### Saving smaller sample for testing
import numpy as np

# Define all species
species_list = ['amecro', 'amerob', 'bewwre', 'bkcchi', 'daejun', 'houfin', 
                'houspa', 'norfli', 'rewbla', 'sonspa', 'spotow', 'whcspa']

# Sample size per class
samples_per_class = 30  # Adjust as needed

# Initialize lists
X = []
y = []

# Loop through species to collect limited data
for i, species in enumerate(species_list):
    dset = f[species]
    print(f"{species}: original shape {dset.shape}")

    available_samples = dset.shape[2]
    use_samples = min(samples_per_class, available_samples)

    X_species = np.transpose(dset[:, :, :use_samples], (2, 0, 1))
    X.append(X_species)
    y.append(np.full(use_samples, i))

# Convert lists to numpy arrays
X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)

### Split into Train Test Split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# train/test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=13)

# Convert target variable to binary class
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

# Check final shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

amecro: original shape (128, 517, 66)
amerob: original shape (128, 517, 172)
bewwre: original shape (128, 517, 144)
bkcchi: original shape (128, 517, 45)
daejun: original shape (128, 517, 125)
houfin: original shape (128, 517, 84)
houspa: original shape (128, 517, 630)
norfli: original shape (128, 517, 37)
rewbla: original shape (128, 517, 187)
sonspa: original shape (128, 517, 263)
spotow: original shape (128, 517, 137)
whcspa: original shape (128, 517, 91)
(241, 128, 517) (241, 12)
(119, 128, 517) (119, 12)


In [13]:
### Optimal Model ---------------------------------------------------------------------
### Define Multi-Class Neural Netowrk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(12, activation='softmax')  # Output layer for multi class
])

### Compile Multi-Class Neural Netowrk
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class loss function
              metrics=['accuracy'])

### Train Multi-Class Neural Network
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=16,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 304ms/step - accuracy: 0.1419 - loss: 65.6166 - val_accuracy: 0.1020 - val_loss: 2.4966
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 300ms/step - accuracy: 0.0667 - loss: 2.6316 - val_accuracy: 0.0816 - val_loss: 2.4869
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 301ms/step - accuracy: 0.1519 - loss: 2.4746 - val_accuracy: 0.1429 - val_loss: 2.4646
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 362ms/step - accuracy: 0.2845 - loss: 2.3718 - val_accuracy: 0.2449 - val_loss: 2.2908
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 376ms/step - accuracy: 0.3193 - loss: 2.1052 - val_accuracy: 0.2857 - val_loss: 2.2665
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 392ms/step - accuracy: 0.4516 - loss: 1.8130 - val_accuracy: 0.2449 - val_loss: 2.7733
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 401ms/step - accuracy: 0.5970 - loss: 1.3980 - val_accuracy: 0.2041 - val_loss: 2.3016
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 374ms/step - accuracy: 0.7395 - loss: 1.0790 - val_accuracy: 0

In [8]:
### Higher dropout ---------------------------------------------------------------------
### Define Multi-Class Neural Netowrk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(12, activation='softmax')  # Output layer for multi class
])

### Compile Multi-Class Neural Netowrk
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class loss function
              metrics=['accuracy'])

### Train Multi-Class Neural Network
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=16,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

C:\Users\nnaga\miniconda3\envs\myenvname\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 223ms/step - accuracy: 0.0902 - loss: 513.6301 - val_accuracy: 0.1020 - val_loss: 2.4829
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - accuracy: 0.1067 - loss: 2.5840 - val_accuracy: 0.1633 - val_loss: 2.4817
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 250ms/step - accuracy: 0.1273 - loss: 2.4701 - val_accuracy: 0.0816 - val_loss: 2.4912
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 249ms/step - accuracy: 0.2355 - loss: 2.3956 - val_accuracy: 0.1429 - val_loss: 2.5280
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - accuracy: 0.3124 - loss: 2.1883 - val_accuracy: 0.1020 - val_loss: 2.4352
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 234ms/step - accuracy: 0.5588 - loss: 1.7150 - val_accuracy: 0.1429 - val_loss: 2.5019
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 233ms/step - accuracy: 0.6456 - loss: 1.1749 - val_accuracy: 0.1837 - val_loss: 2.9178
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 271ms/step - accuracy: 0.7916 - loss: 0.7197 - val_accuracy: 

In [9]:
### More Convolutional Layers ---------------------------------------------------------------------
### Define Multi-Class Neural Netowrk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(12, activation='softmax')  # Output layer for multi class
])

### Compile Multi-Class Neural Netowrk
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class loss function
              metrics=['accuracy'])

### Train Multi-Class Neural Network
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=16,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 377ms/step - accuracy: 0.1026 - loss: 48.2686 - val_accuracy: 0.1020 - val_loss: 2.9885
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 408ms/step - accuracy: 0.0798 - loss: 3.0825 - val_accuracy: 0.1020 - val_loss: 2.4855
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 392ms/step - accuracy: 0.0654 - loss: 2.5116 - val_accuracy: 0.0816 - val_loss: 2.4861
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 386ms/step - accuracy: 0.1088 - loss: 2.4851 - val_accuracy: 0.1224 - val_loss: 2.4863
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 392ms/step - accuracy: 0.0608 - loss: 2.4843 - val_accuracy: 0.0000e+00 - val_loss: 2.4866
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 387ms/step - accuracy: 0.1434 - loss: 2.4822 - val_accuracy: 0.1020 - val_loss: 2.4790
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 393ms/step - accuracy: 0.1568 - loss: 2.4773 - val_accuracy: 0.0204 - val_loss: 2.4872
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 369ms/step - accuracy: 0.1464 - loss: 2.4833 - val_accurac

In [10]:
### Reduced Dropout ---------------------------------------------------------------------
### Define Multi-Class Neural Netowrk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(12, activation='softmax')  # Output layer for multi class
])

### Compile Multi-Class Neural Netowrk
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class loss function
              metrics=['accuracy'])

### Train Multi-Class Neural Network
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=16,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 399ms/step - accuracy: 0.0755 - loss: 59.2262 - val_accuracy: 0.1020 - val_loss: 2.6901
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 403ms/step - accuracy: 0.1048 - loss: 2.7127 - val_accuracy: 0.1633 - val_loss: 2.5380
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 422ms/step - accuracy: 0.1107 - loss: 2.5019 - val_accuracy: 0.0408 - val_loss: 2.4877
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 428ms/step - accuracy: 0.1053 - loss: 2.4865 - val_accuracy: 0.1429 - val_loss: 2.4917
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 444ms/step - accuracy: 0.2108 - loss: 2.4970 - val_accuracy: 0.0408 - val_loss: 2.5640
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 424ms/step - accuracy: 0.1379 - loss: 2.4686 - val_accuracy: 0.0408 - val_loss: 2.5819
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 423ms/step - accuracy: 0.1664 - loss: 2.4751 - val_accuracy: 0.0612 - val_loss: 2.5421
Test Accuracy: 19.33%


In [12]:
### Reduced Dropout, Less layers---------------------------------------------------------------------
### Define Multi-Class Neural Netowrk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 517, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.25),
    Dense(12, activation='softmax')  # Output layer for multi class
])

### Compile Multi-Class Neural Netowrk
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class loss function
              metrics=['accuracy'])

### Train Multi-Class Neural Network
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=16,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: {:.2f}%".format(score[1]*100))

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 208ms/step - accuracy: 0.0994 - loss: 278.4198 - val_accuracy: 0.0816 - val_loss: 2.5720
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - accuracy: 0.1427 - loss: 2.4679 - val_accuracy: 0.1224 - val_loss: 2.4255
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 222ms/step - accuracy: 0.4252 - loss: 2.1908 - val_accuracy: 0.1429 - val_loss: 2.4830
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 209ms/step - accuracy: 0.5930 - loss: 1.6130 - val_accuracy: 0.2041 - val_loss: 2.4605
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - accuracy: 0.8235 - loss: 0.8060 - val_accuracy: 0.1429 - val_loss: 2.7312
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - accuracy: 0.9050 - loss: 0.4969 - val_accuracy: 0.2041 - val_loss: 3.0407
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 236ms/step - accuracy: 0.9555 - loss: 0.2292 - val_accuracy: 0.2653 - val_loss: 3.5737
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 262ms/step - accuracy: 0.9583 - loss: 0.0900 - val_accuracy: 